In [1]:
import pandas as pd

In [2]:
id_ = '27'

In [3]:
df = pd.read_csv(f'Ready_corona_tweets_{id_}.csv')
sent = pd.read_csv(f'corona_tweets_{id_}.csv',header=None)

In [4]:
sent

,0,1
0,1249931432002064385,0.000000
1,1249931432493023237,0.100000
2,1249931435600822272,-0.097222
3,1249931438868193281,0.000000
4,1249931439367319553,0.200000
...,...,...
515734,1250316409738944513,0.000000
515735,1250316411211153411,0.000000
515736,1250316411525726209,0.000000
515737,1250316412062560257,0.400000


In [5]:
statecode = {'AL': 'Alabama',
 'AK': 'Alaska',
 'AZ': 'Arizona',
 'AR': 'Arkansas',
 'CA': 'California',
 'CO': 'Colorado',
 'CT': 'Connecticut',
 'DE': 'Delaware',
 'FL': 'Florida',
 'GA': 'Georgia',
 'HI': 'Hawaii',
 'ID': 'Idaho',
 'IL': 'Illinois',
 'IN': 'Indiana',
 'IA': 'Iowa',
 'KS': 'Kansas',
 'KY': 'Kentucky',
 'LA': 'Louisiana',
 'ME': 'Maine',
 'MD': 'Maryland',
 'MA': 'Massachusetts',
 'MI': 'Michigan',
 'MN': 'Minnesota',
 'MS': 'Mississippi',
 'MO': 'Missouri',
 'MT': 'Montana',
 'NE': 'Nebraska',
 'NV': 'Nevada',
 'NH': 'New Hampshire',
 'NJ': 'New Jersey',
 'NM': 'New Mexico',
 'NY': 'New York',
 'NC': 'North Carolina',
 'ND': 'North Dakota',
 'OH': 'Ohio',
 'OK': 'Oklahoma',
 'OR': 'Oregon',
 'PA': 'Pennsylvania',
 'RI': 'Rhode Island',
 'SC': 'South Carolina',
 'SD': 'South Dakota',
 'TN': 'Tennessee',
 'TX': 'Texas',
 'UT': 'Utah',
 'VT': 'Vermont',
 'VA': 'Virginia',
 'WA': 'Washington',
 'WV': 'West Virginia',
 'WI': 'Wisconsin',
 'WY': 'Wyoming'}

In [6]:
from flashgeotext.geotext import GeoText as g2
from geotext import GeoText
geo = g2(use_demo_data=True)
def extract_country(text):
    if 'america' in str(text).lower():
        return 'US'
    if 'england' in str(text).lower():
        return 'GB'
    if 'ontario' in str(text).lower():
        return 'CA'
    if 'melbourne' in str(text).lower():
        return 'AU'
    places = GeoText(str(text))
    if len(places.country_mentions) == 1:
        for i, j in reversed(places.country_mentions.items()):
            return i
    else:
        try:
            out = list(geo.extract(input_text=text,span_info=False)['countries'].keys())
            if len(out) == 1:
                places = GeoText(str(out[0]))
                if len(places.country_mentions) != 0:
                    for i, j in reversed(places.country_mentions.items()):
                        return i
        except:
            return -1

    return -1

2020-08-11 23:06:37.257 | DEBUG    | flashgeotext.lookup:add:194 - cities added to pool
2020-08-11 23:06:37.263 | DEBUG    | flashgeotext.lookup:add:194 - countries added to pool
2020-08-11 23:06:37.263 | DEBUG    | flashgeotext.lookup:_add_demo_data:225 - demo data loaded for: ['cities', 'countries']


In [7]:
def extract_city(text):
    places = GeoText(str(text))
    if len(places.cities) != 0:
            return places.cities[-1]
    else:
        try:
            out = list(geo.extract(input_text=text,span_info=False)['cities'].keys())[-1]
            places = GeoText(str(out))
            if len(places.cities) != 0:
                    return places.cities[-1]
        except:
            return -1

    return -1

In [8]:
df = df.drop_duplicates()
sent = sent.drop_duplicates()
df = df.join(sent.set_index(0),on='id',how='inner')
df = df.rename(columns={1:'sentiment'})
df = df[['id','created_at','text','user_screen_name','user_location','sentiment']]
df['Date'] = df['created_at'].str.extract(r'.(Mar [0-9]+)')
df['Time'] = df['created_at'].str.extract(r'.Mar [0-9]+.([0-9]+:[0-9]+)')
df['Country'] = df['user_location'].apply(lambda x : extract_country(x))
df['City'] = df['user_location'].apply(lambda x : extract_city(x))
df['cleaned_text'] = df['text'].str.replace(r'(@[a-zA-Z0-9:_]+)','')
df['cleaned_text'] = df['cleaned_text'].str.replace(r'RT','')
df['cleaned_text'] = df['cleaned_text'].str.replace(r'(https[a-zA-Z0-9\/\._:]+)','')
df['Country'] = df.Country.astype(str)
df['City'] = df.City.astype(str)
df.loc[df['Country'] == '-1','Country'] = df.loc[ df['Country'] == '-1','City'].apply(lambda x : extract_country(x))
df.to_csv(f'cleaned{id_}.csv',index=False)

In [9]:
cleaned = pd.read_csv(f'cleaned{id_}.csv')

In [10]:
cleaned

,id,created_at,text,user_screen_name,user_location,sentiment,Date,Time,Country,City,cleaned_text
0,1249931432493023237,Tue Apr 14 05:24:10 +0000 2020,@HackneyAbbott Protocol for treating #COVID201...,THINK3MIND,world,0.100000,NaN,NaN,-1,-1,Protocol for treating #COVID2019 patients &am...
1,1249931460343029760,Tue Apr 14 05:24:17 +0000 2020,RT @naalaYUCK: #ModiMangalMessage \n*Modiji do...,Mamtasi60823339,NaN,0.100000,NaN,NaN,-1,-1,#ModiMangalMessage \n*Modiji does naman and ...
2,1249931449987465218,Tue Apr 14 05:24:14 +0000 2020,RT @KevinMKruse: For the fiscal year ending in...,sutrofog,San Francisco,0.000000,NaN,NaN,US,San Francisco,"For the fiscal year ending in Sept 2006, the..."
3,1249931454659743746,Tue Apr 14 05:24:16 +0000 2020,@Arun2981 Govt can check if you are in an area...,Kakashi0367,"New Delhi, India",0.040179,NaN,NaN,IN,New Delhi,Govt can check if you are in an area with Cor...
4,1249931454328401921,Tue Apr 14 05:24:15 +0000 2020,queen of speaking facts only HA https://t.co/r...,sehunasigoreng,NaN,0.000000,NaN,NaN,-1,-1,queen of speaking facts only HA
...,...,...,...,...,...,...,...,...,...,...,...
414093,1250316407151005698,Wed Apr 15 06:53:55 +0000 2020,RT @Eatala_Rajender: Corona active cases in TS...,shantihp,"Hyderabad,India",-0.133333,NaN,NaN,IN,Hyderabad,Corona active cases in TS 14-04-2020.
414094,1250316386057850885,Wed Apr 15 06:53:50 +0000 2020,@washingtonpost The Defensive strategy against...,Dilawar41,NaN,0.000000,NaN,NaN,-1,-1,The Defensive strategy against corona or COVI...
414095,1250316386292781056,Wed Apr 15 06:53:50 +0000 2020,Right October 12 🥳 https://t.co/mZGNpfZWRF,solesickk,United States,0.285714,NaN,NaN,US,-1,Right October 12 🥳
414096,1250316390654816256,Wed Apr 15 06:53:51 +0000 2020,RT @cheezyboiz: heard one of the symptoms of t...,kitomiez,crys,0.500000,NaN,NaN,-1,-1,heard one of the symptoms of the corona viru...


In [11]:
def tmp_func(x):
    if x != '-1': 
        return city_to_state_dict[x] 

In [12]:
usa = cleaned[cleaned['Country'] == 'US']
from city_to_state import city_to_state_dict


In [13]:
usa.shape

(51195, 11)

In [14]:
city_to_state_dict['Long Island'] = 'New York'
city_to_state_dict['Waco'] = 'Texas'
city_to_state_dict['Laredo'] = 'Texas'
city_to_state_dict['Plano'] = 'Texas'
city_to_state_dict['Vallejo'] = 'California'
city_to_state_dict['Ventura'] = 'California'
city_to_state_dict['Salt Lake'] = 'Utah'
city_to_state_dict['Tyler'] = 'Texas'
city_to_state_dict['Victoria'] = 'Texas'
city_to_state_dict['Killeen'] = 'Texas'
city_to_state_dict['McAllen'] = 'Texas'
city_to_state_dict['Kansas'] = 'Kansas'
city_to_state_dict['New Haven'] = 'Connecticut'
city_to_state_dict['Michigan'] = 'Michigan'
city_to_state_dict['Hampton'] = 'Virginia'
city_to_state_dict['Virginia Beach'] = 'Virginia'
city_to_state_dict['Charlottesville'] = 'Virginia'
city_to_state_dict['Virginia'] = 'Virginia'
city_to_state_dict['Pearland'] = 'Texas'
city_to_state_dict['New York'] = 'New York'
city_to_state_dict['San Antonio'] = 'Texas'
city_to_state_dict['Waxahachie'] = 'Texas'
city_to_state_dict['Texas'] = 'Texas'
city_to_state_dict['Fairfax'] = 'Virginia'
city_to_state_dict['Oklahoma'] = 'Oklahoma'
city_to_state_dict['Iowa'] = 'Iowa'
city_to_state_dict['Carolina'] = 'North Carolina'
city_to_state_dict['Laguna Hills'] = 'California'
city_to_state_dict['League City'] = 'Texas'
city_to_state_dict['Enfield'] = 'Connecticut'
city_to_state_dict['Missouri'] = 'Missouri'
city_to_state_dict['Temple'] = 'Texas'
city_to_state_dict['Maryland'] = 'Maryland'
city_to_state_dict['Visalia'] = 'California'
city_to_state_dict['Danbury'] = 'Connecticut'
city_to_state_dict['La Jolla'] = 'California'
city_to_state_dict['New Braunfels'] = 'Texas'
city_to_state_dict['Nederland'] = 'Texas'
city_to_state_dict['Norwich'] = 'Connecticut'
city_to_state_dict['Lemoore'] = 'California'
city_to_state_dict['Vincent'] = 'California'
city_to_state_dict['Laguna'] = 'California'
city_to_state_dict['Suffolk'] = 'New York'
city_to_state_dict['Victorville'] = 'California'
city_to_state_dict['Lemon Grove'] = 'California'
city_to_state_dict['Morgantown'] = 'West Virginia'
city_to_state_dict['Chesapeake'] = 'Virginia'
city_to_state_dict['Hartford'] = 'Connecticut'
city_to_state_dict['Weslaco'] = 'Texas'
city_to_state_dict['Fredericksburg'] = 'Virginia'
city_to_state_dict['Mechanicsville'] = 'Virginia'
city_to_state_dict['Petersburg'] = 'Virginia'
city_to_state_dict['Roanoke'] = 'Virginia'
city_to_state_dict['Meriden'] = 'Connecticut'
city_to_state_dict['Stamford'] = 'Connecticut'
city_to_state_dict['Stratford'] = 'Connecticut'
city_to_state_dict['Of'] = None
city_to_state_dict['Trumbull'] = 'Connecticut'
city_to_state_dict['Spring'] = None
city_to_state_dict['Newport News'] = 'Rhode Island'
city_to_state_dict['San Juan'] = 'California'
city_to_state_dict['The Woodlands'] = 'Texas'
city_to_state_dict['Sugar Land'] = 'Texas'
city_to_state_dict['Missouri City'] = 'Texas'
city_to_state_dict['Florida'] = 'Florida'
city_to_state_dict['Asia'] = None

In [15]:
usa = usa[usa['City'] !='Paris']

In [16]:
usa = usa[usa['City'] !='Palestine']

In [17]:
usa = usa[usa['City'] !='Vienna']

In [18]:
usa = usa[usa['City'] !='Mission']

In [19]:
usa = usa[usa['City'] !='Lagos']

In [20]:
sentiment = usa.groupby(['City'])['sentiment'].mean().sort_values()
delete_city = sentiment[usa['City'].value_counts() < 50].index

In [21]:
usa = usa[~usa['City'].isin(delete_city)]

In [22]:
usa

,id,created_at,text,user_screen_name,user_location,sentiment,Date,Time,Country,City,cleaned_text
2,1249931449987465218,Tue Apr 14 05:24:14 +0000 2020,RT @KevinMKruse: For the fiscal year ending in...,sutrofog,San Francisco,0.000000,NaN,NaN,US,San Francisco,"For the fiscal year ending in Sept 2006, the..."
30,1249931477200011264,Tue Apr 14 05:24:21 +0000 2020,Breaking out this jem.\n#coronavirus #capcom #...,coug122,"Ohio, USA",0.000000,NaN,NaN,US,-1,Breaking out this jem.\n#coronavirus #capcom #...
77,1249931516890644481,Tue Apr 14 05:24:30 +0000 2020,RT @KevinMKruse: For the fiscal year ending in...,susannajarian,"Seattle, WA",0.000000,NaN,NaN,US,Seattle,"For the fiscal year ending in Sept 2006, the..."
82,1249931521588330497,Tue Apr 14 05:24:32 +0000 2020,RT @KevinMKruse: For the fiscal year ending in...,mattdotgif,"Chicago, IL",0.000000,NaN,NaN,US,Chicago,"For the fiscal year ending in Sept 2006, the..."
152,1249931494447030272,Tue Apr 14 05:24:25 +0000 2020,How to lead in times of corona. https://t.co/S...,rmaxio,"Amsterdam, The Netherlands",0.000000,NaN,NaN,US,Amsterdam,How to lead in times of corona.
...,...,...,...,...,...,...,...,...,...,...,...
414035,1250316385328050176,Wed Apr 15 06:53:50 +0000 2020,"RT @Likeshesays: Michael Avanatti 49, released...",Robbie_Mac,United States,-0.100000,NaN,NaN,US,-1,"Michael Avanatti 49, released from jail over..."
414054,1250316366717931521,Wed Apr 15 06:53:46 +0000 2020,"If it wasn't for Twitter clips, absolutely no ...",realTrumpRallys,USA,-0.300000,NaN,NaN,US,-1,"If it wasn't for Twitter clips, absolutely no ..."
414071,1250316388603838467,Wed Apr 15 06:53:51 +0000 2020,RT @miccxaa: CORONA I JUST WANTED MY DRIVING L...,Lorxenaa,United States,0.000000,NaN,NaN,US,-1,CORONA I JUST WANTED MY DRIVING LICENSE . 😤😤😤
414088,1250316390990401538,Wed Apr 15 06:53:52 +0000 2020,This is abuse https://t.co/yh10ve0Mmq,AllHailChill,"Dallas, TX",0.000000,NaN,NaN,US,Dallas,This is abuse


In [23]:
usa['State'] = usa['City'].apply(lambda x : tmp_func(x))
usa['State'].value_counts()

Texas             4620
New York          3463
California        3393
Illinois          2370
Georgia           1520
Florida           1254
North Carolina     942
Michigan           903
Maryland           882
Washington         810
Pennsylvania       753
Ohio               684
Massachusetts      630
Minnesota          609
Louisiana          512
Arizona            450
Nevada             447
Tennessee          424
Colorado           365
Oregon             308
New Jersey         253
Kansas             253
Missouri           246
New Hampshire      227
Oklahoma           223
Alabama            146
Indiana            143
Iowa               143
Nebraska           133
New Mexico         108
Wisconsin           99
Rhode Island        79
Arkansas            77
Utah                71
Virginia            66
South Carolina      63
Mississippi         60
Connecticut         56
Hawaii              53
Name: State, dtype: int64

In [24]:
usa = usa.reset_index()

In [25]:
states = ['Texas', 'California', 'New York', 'Illinois', 'Florida', 'Georgia',
       'North Carolina', 'Maryland', 'Ohio', 'Michigan', 'Washington',
       'Pennsylvania', 'Louisiana', 'Massachusetts', 'New Hampshire',
       'Minnesota', 'Colorado', 'Nevada', 'Arizona', 'Alabama', 'Tennessee',
       'Oregon', 'New Jersey', 'Kansas', 'Oklahoma', 'Connecticut',
       'Rhode Island', 'Indiana', 'Virginia', 'Nebraska', 'Missouri', 'Iowa',
       'Mississippi', 'Arkansas', 'Wisconsin', 'South Carolina', 'New Mexico',
       'Kentucky', 'Hawaii', 'Utah', 'Idaho', 'Alaska', 'Vermont', 'Montana',
       'Delaware', 'West Virginia']
for i in range(len(usa)):
    for j in states:
        if j.lower() in usa.loc[i,'user_location'].lower():
            usa.loc[i,'State'] = j
    

In [26]:
for i in range(len(usa)):
    for j in statecode:
        if j in usa.loc[i,'user_location']:
            usa.loc[i,'State'] = statecode[j]

In [27]:
usa['State'].value_counts()

Texas             4924
California        4851
New York          3656
Washington        1820
Illinois          1502
Georgia           1355
Florida           1279
Pennsylvania      1069
Ohio               896
Michigan           868
Maryland           823
Massachusetts      795
North Carolina     777
Arizona            745
Tennessee          625
Louisiana          604
New Jersey         572
Oregon             548
Nevada             516
Minnesota          398
Missouri           374
Indiana            364
South Carolina     333
Alabama            297
Wisconsin          287
New Hampshire      282
Colorado           258
Kentucky           248
Oklahoma           224
Connecticut        217
Mississippi        209
Kansas             170
Iowa               155
Nebraska           155
Rhode Island       144
Utah               140
Arkansas           123
New Mexico         113
Hawaii             113
Virginia            89
Delaware            66
West Virginia       65
Idaho               44
Alaska     

In [28]:
usa['user_screen_name'].value_counts()

jamesbaneny       136
LSchaffer16        82
johnson7777777     64
mapsofworld        61
rtehrani           56
                 ... 
inmyworstlies       1
dbubna              1
ashleyvenom         1
TruthOverBliss      1
Evelyn53417888      1
Name: user_screen_name, Length: 33296, dtype: int64

In [29]:
usa.to_csv(f'usa{id_}.csv',index=None)

In [30]:
usa.shape

(39714, 13)

In [31]:
usa.sample(20)

,index,id,created_at,text,user_screen_name,user_location,sentiment,Date,Time,Country,City,cleaned_text,State
35205,366193,1250274948150509573,Wed Apr 15 04:09:11 +0000 2020,RT @21savage: Bang outside I hang outside \ndo...,Camaaveli,"Omaha, NE",0.000,NaN,NaN,US,Omaha,Bang outside I hang outside \ndon’t come out...,Nebraska
39442,409692,1250312951933722624,Wed Apr 15 06:40:12 +0000 2020,RT @21savage: Bang outside I hang outside \ndo...,hlav__,United States,0.000,NaN,NaN,US,-1,Bang outside I hang outside \ndon’t come out...,None
8131,148620,1250060476010070016,Tue Apr 14 13:56:57 +0000 2020,Spoiler alert: it’s me https://t.co/h5vtcoeJQx,seanlynchpoet,"Philadelphia, PA",0.000,NaN,NaN,US,Philadelphia,Spoiler alert: it’s me,Pennsylvania
38707,399388,1250302280605126656,Wed Apr 15 05:57:47 +0000 2020,I miss you but I can’t come to u I’m getting t...,anasakrane,"New York, NY",-0.400,NaN,NaN,US,New York,I miss you but I can’t come to u I’m getting t...,New York
31975,343516,1250256282453512195,Wed Apr 15 02:55:01 +0000 2020,RT @21savage: Bang outside I hang outside \ndo...,kelrencoo,"Michigan, USA",0.000,NaN,NaN,US,Michigan,Bang outside I hang outside \ndon’t come out...,Michigan
12331,195496,1250095220911493120,Tue Apr 14 16:15:00 +0000 2020,RT @KevinMKruse: For the fiscal year ending in...,BeyondTolaria,"Louisiana, USA",0.000,NaN,NaN,US,-1,"For the fiscal year ending in Sept 2006, the...",Louisiana
22371,286009,1250186824171098114,Tue Apr 14 22:19:00 +0000 2020,"RT @no_silenced: Hey Liberals, you can’t suppo...",SandraQ21416819,United States,0.000,NaN,NaN,US,-1,"Hey Liberals, you can’t support them forcing...",None
13878,212935,1250108240509181954,Tue Apr 14 17:06:45 +0000 2020,RT @cjtruth: GOOD MORNING PATRIOTS! The war is...,SevenStAndre1,"Texas, USA",0.725,NaN,NaN,US,Texas,GOOD MORNING PATRIOTS! The war is not over b...,Texas
30966,336936,1250252020101103617,Wed Apr 15 02:38:04 +0000 2020,RT @21savage: Bang outside I hang outside \ndo...,Kikz369_,"Brooklyn, NY",0.000,NaN,NaN,US,Brooklyn,Bang outside I hang outside \ndon’t come out...,New York
8055,147894,1250059928179478534,Tue Apr 14 13:54:46 +0000 2020,RT @PhysohMadness: Corona virus taught me that...,aynek3_,Baltimore,0.500,NaN,NaN,US,Baltimore,Corona virus taught me that..\n\n1. We need ...,Maryland
